<a href="https://colab.research.google.com/github/mnmahir/best-fyp/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Import Libraries**

In [ ]:
# Google Drive
from google.colab import drive
# General
import math
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
# Keras Library


print("Keras version: ", keras.__version__)
print("TensorFlow version:", tf.__version__)

#**Mounting drive and setting path**

In [ ]:
drive.mount('/content/drive')

In [ ]:
# Path where dataset is located
dataset_path = '/content/drive/My Drive/FYP Stuff/Datasets/#old/Dataset 1C'
train_path = dataset_path + '/Train'
valid_path = dataset_path + '/Test'    # Some literature use the term "test set" / "valid set" but it is actually a "dev set". This is the set that is iterated during training.
#test_path = dataset_path + '/Test'  # Test set helps evaluate how good your final system is. It's ok not to have test set.

#**Variable Initialization**

In [ ]:
IMAGE_CLASSES = ['Grade I','Grade II','Grade III', 'Normal']         # Classes (Name must be exactly the same as folder name in each set)
IMAGE_SIZE = [224,224]          # Image size (Set for resizing later)
BATCH_SIZE = 16                 # Mini-batch size
SEED = 1                        # Fixed randomness

# **Initialize Functions**
**Data Augmentation**

In [ ]:
############################
# Training data generator
############################
def get_train_generator():
  print("Getting train generator...")
  image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range = 90
  )
  
  generator = image_generator.flow_from_directory(
      batch_size = BATCH_SIZE,
      directory = train_path,
      shuffle = True,             # Shuffle input after each epoch
      target_size = IMAGE_SIZE,   # Set image size to be [x]px by [y]px set in IMAGE_SIZE
      class_mode = 'categorical',
      seed = SEED,
  )

  return generator
############################
# Validation data generator
############################
def get_valid_generator():
  print("Getting valid generator...")
  image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255
  )
  
  generator = image_generator.flow_from_directory(
      batch_size = BATCH_SIZE,
      directory = valid_path,
      shuffle = False,             # Shuffle input after each epoch
      target_size = IMAGE_SIZE,   # Set image size to be [x]px by [y]px set in IMAGE_SIZE
      class_mode = 'categorical',
      seed = SEED,
  )

  return generator

In [ ]:
train_generator = get_train_generator()
valid_generator = get_valid_generator()

In [ ]:
# Solving class imbalance
def get_freq_data(generator):
  class_data_labels = generator.classes
  total_class = len(IMAGE_CLASSES)
  freq_out = []

  for i in range(total_class):
    total_label_i = 0
    for ele in class_data_labels:
      if ele == i:
        total_label_i += 1
    freq_out.append(total_label_i)
  
  print("Total number of class is:", total_class)
  print("Total image in generator is: ", freq_out)


In [ ]:
get_freq_data(valid_generator)

In [ ]:
def show_image_generated(generator, index, img_size):
  x, y = valid_generator.__getitem__(index)
  img_plt = math.ceil(math.sqrt(BATCH_SIZE))
  grid_plt = str(img_plt)+str(img_plt)+str(1)
  plt.figure(1, figsize = (img_size,img_size))
  for i in range(BATCH_SIZE):
    plt.subplot(img_plt,img_plt,i+1)
    plt.imshow(x[i])
    plt.title(IMAGE_CLASSES[np.where(y[i] == 1.)[0][0]], fontsize = 2+img_size, pad = 0)
    plt.axis('off')
  plt.show()

In [ ]:
show_image_generated(train_generator, index = 0, img_size = 5)  # Print image generated by generator. Change index to see different batch (start at 0)

# **Model Development**

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

for layer in base_model.layers:
  layer.trainable = False

x = tf.keras.layers.Flatten()(base_model.output)

prediction = tf.keras.layers.Dense(len(IMAGE_CLASSES), activation = 'softmax')(x)

model = tf.keras.Model(inputs = base_model.input, outputs = prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
history = model.fit(
  train_generator,
  validation_data=valid_generator,
  epochs=3,
  steps_per_epoch=len(train_generator),
  validation_steps=len(valid_generator)
)